In [1]:
# import dependencies / modules

import time
import re
from sqlalchemy import create_engine
import json
import pandas as pd
import numpy as np
import psycopg2
from config import db_password

In [2]:
# Define variable for database connection
protocol = "postgres"
user = "postgres"
location = "localhost"
port = "5432"
db = "movie_data"

In [3]:
connection = f'{protocol}://{user}:{db_password}@{location}:{port}/{db}'
engine = create_engine(connection)


In [4]:
def clean_movie(movie): # Create a function for cleaning. This is unique to wiki_file only. 
    
    movie = dict(movie) #create a non-destructive copy of movie
  
    alt_titles = {}     #Create an empty list for alt_title 
                        # combine alternate titles into one list
        
    for key in ['Also known as','Arabic','Cantonese','Chinese','French',
                'Hangul','Hebrew','Hepburn','Japanese','Literally',
                'Mandarin','McCune-Reischauer','Original title','Polish',
                'Revised Romanization','Romanized','Russian',
                'Simplified','Traditional','Yiddish']:
        
        if key in movie:
            alt_titles[key] = movie[key]
            movie.pop(key)
   
    if len(alt_titles) > 0:    # Use the length of alt_titles to check or validate value
        movie['alt_titles'] = alt_titles  # Convert to alt_titles i.e combine alternate title into one list

    # merge column names
    def change_column_name(old_name, new_name):  # Create a function change_column_name old and new 
        
        if old_name in movie:
            movie[new_name] = movie.pop(old_name)   # Use pop() method to consolidate columns with
                                                    # same data into one or change the name of a key
   
    change_column_name('Adaptation by', 'Writer(s)') # Go through and decide if there’s a better name for it.
    change_column_name('Country of origin', 'Country')
    change_column_name('Directed by', 'Director')
    change_column_name('Distributed by', 'Distributor')
    change_column_name('Edited by', 'Editor(s)')
    change_column_name('Length', 'Running time')
    change_column_name('Original release', 'Release date')
    change_column_name('Music by', 'Composer(s)')
    change_column_name('Produced by', 'Producer(s)')
    change_column_name('Producer', 'Producer(s)')
    change_column_name('Productioncompanies ', 'Production company(s)')
    change_column_name('Productioncompany ', 'Production company(s)')
    change_column_name('Released', 'Release Date')
    change_column_name('Release Date', 'Release date')
    change_column_name('Screen story by', 'Writer(s)')
    change_column_name('Screenplay by', 'Writer(s)')
    change_column_name('Story by', 'Writer(s)')
    change_column_name('Theme music composer', 'Composer(s)')
    change_column_name('Written by', 'Writer(s)')

    return movie


def etl_pipe_func(wiki_file, kaggle_file, ratings_file):
        
    # Open/Load and read the first file, a .json raw file, using "with open()" 
    with open(f'{file_dir}/wikipedia.movies.json', mode='r') as file:
        wiki_movies_raw = json.load(file)
    
    # Read the second file, a csv file, using Pandas pd.read_csv() function    
    kaggle_metadata = pd.read_csv(f'{file_dir}/movies_metadata.csv')

    # Read the third file, a csv file, using Pandas pd.read_csv() function
    ratings = pd.read_csv(f'{file_dir}/ratings.csv')

    
    wiki_movies = [movie for movie in wiki_movies_raw
               if ('Director' in movie or 'Directed by' in movie)
                   and 'imdb_link' in movie]
    clean_movies = [clean_movie(movie) for movie in wiki_movies]
    
    wiki_movies_df = pd.DataFrame(clean_movies)
    
    # This is the beginig of cleaning the first file
    # Use imdb_id to check as we clean out duplicates and drop NAs. Extact "imdb_id" from imdb_link
    try:
        wiki_movies_df['imdb_id'] = wiki_movies_df['imdb_link'].str.extract(r'(tt\d{7})')
        wiki_movies_df.drop_duplicates(subset='imdb_id', inplace = True)
    except Exception as exception:
        print('exception')
        
    # Apply list comprehension    
    wiki_columns_to_keep = [column for column in wiki_movies_df.columns if wiki_movies_df[column].isnull().sum() < len(wiki_movies_df) * 0.9]
    wiki_movies_df = wiki_movies_df[wiki_columns_to_keep]
    
    # Drop Nas in wiki_movies Box Office
    box_office = wiki_movies_df['Box office'].dropna() 
    box_office = box_office.apply(lambda x: ' '.join(x) if type(x) == list else x)
    
    # Apply regular expression
    form_one = r'\$\d+\.?\d*\s*[mb]illion' # Create a variable for the first regular expression
                                       # To count up how many box office values match our first form
                                       # Use str.contain() method. Set the flag to ignore case
    form_two = r'\$\d{1,3}(?:,\d{3})+'    
    
    # Parse the Box Office Data to Numeric values
    def parse_dollars(s):
        # if s is not a string, return NaN
        if type(s) != str:
            return np.nan

        # if input is of the form $###.# million
        if re.match(r'\$\s*\d+\.?\d*\s*milli?on', s, flags=re.IGNORECASE):

            # remove dollar sign and " million"
            s = re.sub('\$|\s|[a-zA-Z]','', s)

            # convert to float and multiply by a million
            value = float(s) * 10**6

            # return value
            return value

        # if input is of the form $###.# billion
        elif re.match(r'\$\s*\d+\.?\d*\s*billi?on', s, flags=re.IGNORECASE):
    
            # remove dollar sign and " billion"
            s = re.sub('\$|\s|[a-zA-Z]','', s)

            # convert to float and multiply by a billion
            value = float(s) * 10**9

            # return value
            return value

        # if input is of the form $###,###,###
        elif re.match(r'\$\s*\d{1,3}(?:[,\.]\d{3})+(?!\s[mb]illion)', s, flags=re.IGNORECASE):

            # remove dollar sign and commas
            s = re.sub('\$|,','', s)

            # convert to float
            value = float(s)

            # return value
            return value

        # otherwise, return NaN
        else:
            return np.nan

    
    # First we need to extract the values from Box Office using .str.extract. Then, apply
    # parse_dollars to the first column in the DataFrame returned by .str.extract
    wiki_movies_df['box_office'] = box_office.str.extract(f'({form_one}|{form_two})', flags=re.IGNORECASE)[0].apply(parse_dollars)
    wiki_movies_df.drop('Box office', axis=1, inplace=True)
    
    # Create a budget variable 
    budget = wiki_movies_df['Budget'].dropna()

    # Convert any lists to strings:
    budget = budget.map(lambda x: ' '.join(x) if type(x) == list else x)
    budget = budget.str.replace(r'\$.*[--](?![a-z])', '$', regex=True)
    
    # Parse Budget using the same code as in Box Office. Replace box_office with budget
    wiki_movies_df['budget'] = budget.str.extract(f'({form_one}|{form_two})', flags=re.IGNORECASE)[0].apply(parse_dollars)
    wiki_movies_df.drop('budget', axis=1, inplace=True)
    
    release_date = wiki_movies_df['Release date'].dropna().apply(lambda x: ' '.join(x) if type(x) == list else x)
    
    # Convert dates to forms
    date_form_one = r'(?:January|February|March|April|May|June|July|August|September|October|November|December)\s[123]\d,\s\d{4}'
    date_form_two = r'\d{4}.[01]\d.[123]\d'
    date_form_three = r'(?:January|February|March|April|May|June|July|August|September|October|November|December)\s\d{4}'
    date_form_four = r'\d{4}'
    wiki_movies_df['release_date'] = pd.to_datetime(release_date.str.extract(f'({date_form_one}|{date_form_two}|{date_form_three}|{date_form_four})')[0], infer_datetime_format=True)
    
    # Dropna from Wiki_movies
    running_time = wiki_movies_df['Running time'].dropna().apply(lambda x: ' '.join(x) if type(x) == list else x)
    running_time_extract = running_time.str.extract(r'(\d+)\s*ho?u?r?s?\s*(\d*)|(\d+)\s*m')
    running_time_extract = running_time_extract.apply(lambda col: pd.to_numeric(col, errors='coerce')).fillna(0)
    wiki_movies_df['running_time'] = running_time_extract.apply(lambda row: row[0]*60 + row[1] if row[2] == 0 else row[2], axis=1)
    wiki_movies_df.drop('Running time', axis=1, inplace=True)
    
    kaggle_metadata = kaggle_metadata[kaggle_metadata['adult'] == 'False'].drop('adult',axis='columns')
    kaggle_metadata['video'] = kaggle_metadata['video'] == 'True'
    
    # Convert Kaggle to Numeric values
    kaggle_metadata['budget'] = kaggle_metadata['budget'].astype(int)
    kaggle_metadata['id'] = pd.to_numeric(kaggle_metadata['id'], errors='raise')
    kaggle_metadata['popularity'] = pd.to_numeric(kaggle_metadata['popularity'], errors='raise')
    kaggle_metadata['release_date'] = pd.to_datetime(kaggle_metadata['release_date'])
    
    
    # Merge Wikipedia data and Kaggle data after being cleaned up and in tabular formats
    # The main purpose is to avoid duplicates. We also drop some columns that are redundant.
    movies_df = pd.merge(wiki_movies_df, kaggle_metadata, on='imdb_id', suffixes=['_wiki','_kaggle'])
    movies_df.drop(columns=['title_wiki','release_date_wiki','Language','Production company(s)'], inplace=True)

    def fill_missing_kaggle_data(df, kaggle_column, wiki_column):
        df[kaggle_column] = df.apply(
            lambda row: row[wiki_column] if row[kaggle_column] == 0 else row[kaggle_column]
        , axis=1)
        df.drop(columns=wiki_column, inplace=True)
        
    fill_missing_kaggle_data(movies_df, 'runtime', 'running_time')
    fill_missing_kaggle_data(movies_df, 'budget_kaggle', 'budget_wiki')
    fill_missing_kaggle_data(movies_df, 'revenue', 'box_office')

    # Reorder the columns to make the information easier to read and understand
    movies_df = movies_df.loc[:, ['imdb_id','id','title_kaggle','original_title','tagline','belongs_to_collection','url','imdb_link',
                       'runtime','budget_kaggle','revenue','release_date_kaggle','popularity','vote_average','vote_count',
                       'genres','original_language','overview','spoken_languages','Country',
                       'production_companies','production_countries','Distributor',
                       'Producer(s)','Director','Starring','Cinematography','Editor(s)','Writer(s)','Composer(s)','Based on'
                      ]]
    
    # For consistency we rename the columns in movies DataFrame
    movies_df.rename({'id':'kaggle_id',
                  'title_kaggle':'title',
                  'url':'wikipedia_url',
                  'budget_kaggle':'budget',
                  'release_date_kaggle':'release_date',
                  'Country':'country',
                  'Distributor':'distributor',
                  'Producer(s)':'producers',
                  'Director':'director',
                  'Starring':'starring',
                  'Cinematography':'cinematography',
                  'Editor(s)':'editors',
                  'Writer(s)':'writers',
                  'Composer(s)':'composers',
                  'Based on':'based_on'
                 }, axis='columns', inplace=True)
    
    rating_counts = ratings.groupby(['movieId','rating'], as_index=False).count() \
                .rename({'userId':'count'}, axis=1)
    rating_counts.columns = ['rating_' + str(col) for col in rating_counts.columns]
    
    # Merge the data between files
    movies_with_ratings_df = pd.merge(movies_df, rating_counts, left_on='kaggle_id', right_index=True, how='left')
    
    # Fill in zeros for missing values
    movies_with_ratings_df[rating_counts.columns] = movies_with_ratings_df[rating_counts.columns].fillna(0)
    
    # Define variables for database connection
    protocol = "postgres"
    user = "postgres2020"
    location = "localhost"
    port = "5432"
    db = "movie_data"
    
    # Provide connection string to Postgres db
    connection = f'{protocol}://{user}:{db_password}@{location}:{port}/{db}'
    
    # Create database engine
    engine = create_engine(connection)
    
    # Import Movie data to the sql database
    movies_df.to_sql(name='movies', con=engine)
    
    # create a variable for the number of rows imported
    rows_imported = 0
    for data in pd.read_csv(f'{file_dir}ratings.csv', chunksize=1000000):
    
    # Print out the range of rows that are being imported
        print(f'importing rows {rows_imported} to {rows_imported + len(data)}...', end='')
    
    data.to_sql(name='ratings', con=engine, if_exists='append')
    
    # Increment the number of rows imported by the size of 'data'
    rows_imported += len(data)
    
    # Print that the rows have finished importing
    print('Done importing...........')
    
    # This is the working directory path for all resource files
    file_dir = 'C:/Users/Franc/Berkeley/class_folder/Movies_ETL/Resources'
    
    wiki_file = f'{file_dir}/wikipedia.movies.json'
    kaggle_file = f'{file_dir}/movies_metadata.csv'
    ratings_file = f'{file_dir}/ratings_csv'
    
    # Call the main function
    
    etl_pipe_func(wiki_file, kaggle_file, ratings_file)
    
    